![Credit card being held in hand](credit_card.jpg)

Commercial banks receive _a lot_ of applications for credit cards. Many of them get rejected for many reasons, like high loan balances, low income levels, or too many inquiries on an individual's credit report, for example. Manually analyzing these applications is mundane, error-prone, and time-consuming (and time is money!). Luckily, this task can be automated with the power of machine learning and pretty much every commercial bank does so nowadays. In this workbook, you will build an automatic credit card approval predictor using machine learning techniques, just like real banks do.

### The Data

The data is a small subset of the Credit Card Approval dataset from the UCI Machine Learning Repository showing the credit card applications a bank receives. This dataset has been loaded as a `pandas` DataFrame called `cc_apps`. The last column in the dataset is the target value.

In [41]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

# Load the dataset
cc_apps = pd.read_csv("cc_approvals.data", header=None) 
cc_apps.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,g,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,g,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,g,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,g,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,s,0,+


In [42]:
cc_apps

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,g,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,g,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,g,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,g,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,s,0,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,b,21.08,10.085,y,p,e,h,1.25,f,f,0,g,0,-
686,a,22.67,0.750,u,g,c,v,2.00,f,t,2,g,394,-
687,a,25.25,13.500,y,p,ff,ff,2.00,f,t,1,g,1,-
688,b,17.92,0.205,u,g,aa,v,0.04,f,f,0,g,750,-


In [43]:
#preprocessing dataset to replace characters with numerical values
print(cc_apps.describe())
print(cc_apps.info())
print(cc_apps.dtypes)

               2           7          10             12
count  690.000000  690.000000  690.00000     690.000000
mean     4.758725    2.223406    2.40000    1017.385507
std      4.978163    3.346513    4.86294    5210.102598
min      0.000000    0.000000    0.00000       0.000000
25%      1.000000    0.165000    0.00000       0.000000
50%      2.750000    1.000000    0.00000       5.000000
75%      7.207500    2.625000    3.00000     395.500000
max     28.000000   28.500000   67.00000  100000.000000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       690 non-null    object 
 1   1       690 non-null    object 
 2   2       690 non-null    float64
 3   3       690 non-null    object 
 4   4       690 non-null    object 
 5   5       690 non-null    object 
 6   6       690 non-null    object 
 7   7       690 non-null    float64
 8   8       690 non-

In [44]:
missing_values = ["?", "N/A", "null", "unknown", " ", "NaN", "nan", "NULL", ""]
cc_apps = cc_apps.replace(missing_values, np.nan)
cc_apps_copy = cc_apps.copy()
print(cc_apps_copy)

for column in cc_apps_copy.columns:
    cc_apps_copy[column] = pd.to_numeric(cc_apps_copy[column], errors="ignore")
    
    if cc_apps_copy[column].dtype == "object":
        most_freq = cc_apps_copy[column].value_counts().index[0]
        cc_apps_copy[column].fillna(most_freq, inplace=True)
    else:
        mean_val = cc_apps_copy[column].mean()
        cc_apps_copy[column].fillna(mean_val, inplace=True)

cc_apps_copy

    0      1       2  3  4   5   6     7  8  9   10 11   12 13
0    b  30.83   0.000  u  g   w   v  1.25  t  t   1  g    0  +
1    a  58.67   4.460  u  g   q   h  3.04  t  t   6  g  560  +
2    a  24.50   0.500  u  g   q   h  1.50  t  f   0  g  824  +
3    b  27.83   1.540  u  g   w   v  3.75  t  t   5  g    3  +
4    b  20.17   5.625  u  g   w   v  1.71  t  f   0  s    0  +
..  ..    ...     ... .. ..  ..  ..   ... .. ..  .. ..  ... ..
685  b  21.08  10.085  y  p   e   h  1.25  f  f   0  g    0  -
686  a  22.67   0.750  u  g   c   v  2.00  f  t   2  g  394  -
687  a  25.25  13.500  y  p  ff  ff  2.00  f  t   1  g    1  -
688  b  17.92   0.205  u  g  aa   v  0.04  f  f   0  g  750  -
689  b  35.00   3.375  u  g   c   h  8.29  f  f   0  g    0  -

[690 rows x 14 columns]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,g,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,g,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,g,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,g,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,s,0,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,b,21.08,10.085,y,p,e,h,1.25,f,f,0,g,0,-
686,a,22.67,0.750,u,g,c,v,2.00,f,t,2,g,394,-
687,a,25.25,13.500,y,p,ff,ff,2.00,f,t,1,g,1,-
688,b,17.92,0.205,u,g,aa,v,0.04,f,f,0,g,750,-


In [45]:
cc_apps_final = pd.get_dummies(cc_apps_copy, drop_first=True)
print(cc_apps_final)

         1       2     7  10   12  0_b  ...  6_z  8_t  9_t  11_p  11_s  13_-
0    30.83   0.000  1.25   1    0    1  ...    0    1    1     0     0     0
1    58.67   4.460  3.04   6  560    0  ...    0    1    1     0     0     0
2    24.50   0.500  1.50   0  824    0  ...    0    1    0     0     0     0
3    27.83   1.540  3.75   5    3    1  ...    0    1    1     0     0     0
4    20.17   5.625  1.71   0    0    1  ...    0    1    0     0     1     0
..     ...     ...   ...  ..  ...  ...  ...  ...  ...  ...   ...   ...   ...
685  21.08  10.085  1.25   0    0    1  ...    0    0    0     0     0     1
686  22.67   0.750  2.00   2  394    0  ...    0    0    1     0     0     1
687  25.25  13.500  2.00   1    1    0  ...    0    0    1     0     0     1
688  17.92   0.205  0.04   0  750    1  ...    0    0    0     0     0     1
689  35.00   3.375  8.29   0    0    1  ...    0    0    0     0     0     1

[690 rows x 36 columns]


In [46]:
y = cc_apps_final.iloc[:, -1].values
X = cc_apps_final.iloc[:, :-1].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [47]:
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.neighbors import KNeighborsClassifier

kf = KFold(n_splits=5, shuffle=True, random_state=42)
param_grid = {'n_neighbors': np.arange(1, 21)}  

grid_knn = GridSearchCV(KNeighborsClassifier(), param_grid, cv=kf)
grid_knn.fit(X_train_scaled, y_train)

print(f"Best K for KNN: {grid_knn.best_params_['n_neighbors']}")
print(f"Best cross-validated accuracy for KNN: {grid_knn.best_score_:.4f}")

best_knn = grid_knn.best_estimator_

Best K for KNN: 6
Best cross-validated accuracy for KNN: 0.8075


In [48]:
logreg_params = {'C': np.logspace(-3, 3, 7), 'max_iter': [100, 200, 300]}
grid_logreg = GridSearchCV(LogisticRegression(), logreg_params, cv=kf)
grid_logreg.fit(X_train_scaled, y_train)

print(f"Best params for Logistic Regression: {grid_logreg.best_params_}")
print(f"Best cross-validated accuracy for Logistic Regression: {grid_logreg.best_score_:.4f}")

best_logreg = grid_logreg.best_estimator_


Best params for Logistic Regression: {'C': 1.0, 'max_iter': 100}
Best cross-validated accuracy for Logistic Regression: 0.8675


In [49]:
from sklearn.tree import DecisionTreeClassifier
models = {
    "Logistic Regression": best_logreg,
    "KNN": best_knn,
    "Decision Tree": DecisionTreeClassifier()
}

model_scores = {}

for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)

    acc = model.score(X_test_scaled, y_test)
    cm = confusion_matrix(y_test, y_pred)

    model_scores[name] = acc

    print(f"--- {name} ---")
    print(f"Accuracy: {acc:.4f}")
    print(f"Confusion Matrix:\n{cm}\n")


--- Logistic Regression ---
Accuracy: 0.8164
Confusion Matrix:
[[81 16]
 [22 88]]

--- KNN ---
Accuracy: 0.7923
Confusion Matrix:
[[76 21]
 [22 88]]

--- Decision Tree ---
Accuracy: 0.8019
Confusion Matrix:
[[73 24]
 [17 93]]



In [50]:
best_model_name = max(model_scores, key=model_scores.get)
best_model = models[best_model_name]
best_score = best_model.score(X_test_scaled, y_test)
print(best_score)

0.8164251207729468
